In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import DataCollatorForCompletionOnlyLM, SFTConfig, SFTTrainer

In [ ]:
model_id = "HuggingFaceTB/SmolLM-135M-Instruct"
dataset_id = "medalpaca/medical_meadow_medical_flashcards"
device = "cuda" if torch.cuda.is_available() else "cpu"

## Preparing and Formatting the Dataset for Training 

We'll be preparing a dataset for training a Language Model (LLM). The steps involve formatting the dataset to keep only the necessary columns and splitting it into training and evaluation sets. Proper dataset preparation is crucial for ensuring the model's effectiveness and generalization.

In [ ]:
def format_dataset(dataset, keys, instruction_col_name, response_col_name):
    """Format the dataset by retaining only necessary columns and renaming them."""
    cols_to_remove = [key for key in keys if key not in [instruction_col_name, response_col_name]]
    dataset = dataset.remove_columns(cols_to_remove)
    dataset = dataset.rename_column(instruction_col_name, "instruction")
    dataset = dataset.rename_column(response_col_name, "response")
    return dataset
    
def prepare_datasets(dataset, instruction_col_name, response_col_name):
    """Format and split the dataset for training and evaluation."""
    available_cols = list(dataset["train"].features.keys())
    formatted_dataset = format_dataset(
        dataset, available_cols, instruction_col_name, response_col_name
    )

    if "valid" in formatted_dataset:
        train_dataset = formatted_dataset["train"]
        eval_dataset = formatted_dataset["valid"]
    elif "test" in formatted_dataset:
        train_dataset = formatted_dataset["train"]
        eval_dataset = formatted_dataset["test"]
    else:
        split_dataset = formatted_dataset["train"].train_test_split(test_size=0.2)
        train_dataset, eval_dataset = split_dataset["train"], split_dataset["test"]

    return train_dataset, eval_dataset

load the dataset using its ID or path. This dataset will be used for training and evaluating the model.

In [ ]:
dataset = load_dataset(dataset_id)

Print the dataset information to inspect its structure and column names. This is important to understand the data we're working with and ensure that we correctly identify the columns containing the `instructions` and `responses`.

In [ ]:
dataset

In [ ]:
dataset["train"][0]

Format the dataset and split it into training and evaluation sets. Here, `input` and `output` represent the columns in the dataset holding the `instruction` and `response`, respectively.

In [ ]:
train_dataset, eval_dataset = prepare_datasets(
    dataset, instruction_col_name="input", response_col_name="output"
)

Ensures that all previous operations have been executed as intended.

In [ ]:
print(f"{train_dataset = }")
print(f"{eval_dataset = }")

## Load and Test Pre-trained Model

Define Functions for Response Generation and Display

In [ ]:
def generate_response(model, tokenizer, instruction, device="cpu"):
    """Generate a response from the model based on an instruction."""
    messages = [{"role": "user", "content": instruction}]
    input_text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
    outputs = model.generate(
        inputs, max_new_tokens=128, temperature=0.2, top_p=0.9, do_sample=True
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

def print_example(example):
    """Print an example from the dataset."""
    print(f"Original Dataset Example:")
    print(f"Instruction: {example['instruction']}")
    print(f"Response: {example['response']}")
    print("-" * 100)

def print_response(response):
    """Print the model's response."""
    print(f"Model response:")
    print(response.split("assistant\n")[-1])
    print("-" * 100)

Load the Model, and the Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

#### Test the Pre-trained Model

In [ ]:
# Define a test example
example1 = eval_dataset[1]

response = generate_response(model, tokenizer, example1["instruction"], device)

print_example(example1)
print_response(response)

### Supervised Fine-tuning Trainer

#### Training adapters [Read More](https://huggingface.co/docs/trl/v0.9.6/en/sft_trainer#training-adapters)

Huggingface support tight integration with 🤗 PEFT library so that we can conveniently train adapters instead of training the entire model.

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none"
)

#### Customize prompts using packed dataset [Read More](https://huggingface.co/docs/trl/en/sft_trainer#customize-your-prompts-using-packed-dataset)

Since our dataset has two field `instruction` and `response`, we need to combine them as one string to be able to past it to the SFT Trainer.

In [ ]:
def formatting_prompts_func(example: dict) -> str:
    """Format prompt for training."""
    text = f"<|im_start|>user\n{example['instruction']}<|im_end|>\n<|im_start|>assistant\n{example['response']}<|im_end|>"
    return text

In [ ]:
num_train_epochs = 5

output_dir = f"{model_id.split('/')[-1]}-{dataset_id.split('/')[-1]}-{num_train_epochs}epochs"

sft_config = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    max_seq_length=512,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    save_steps=500,  # save checkpoints every n training steps
    logging_steps=500,
    learning_rate=1e-3,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    warmup_ratio=0.05,
    lr_scheduler_type="constant",
    packing=True
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    formatting_func=formatting_prompts_func,
    peft_config=peft_config,
    args=sft_config,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model()

#### Test Fine-tuned Model

In [ ]:
ft_model = AutoModelForCausalLM.from_pretrained(output_dir).to(device)

response = generate_response(ft_model, tokenizer, example1["instruction"], device)

print_example(example1)
print_response(response)

#### Push the fine-tuned model to your HuggingFace

In [ ]:
hf_access_token = ""
if hf_access_token:
    trainer.push_to_hub(token=hf_access_token)